In [6]:
import torch 

print(torch.cuda.device_count())
print(torch.device('cuda'))


0
cuda


In [7]:
# Testing Blackjack agent by
import gymnasium as gym
from blackjack_agent_package import blackjack_agent_class as BJA
from blackjack_agent_package import Q_value_table_class as QVT

# hyperparameters
learning_rate = 0.01
n_episodes = 1000
start_epsilon = 1.0
epsilon_decay = start_epsilon / (n_episodes / 2)  # reduce the exploration over time
final_epsilon = 0.1

env = gym.make("Blackjack-v1", sab=False)
# env = gym.wrappers.RecordEpisodeStatistics(env, buffer_length=n_episodes)
action_size = env.action_space.n

Q_value_table = QVT.QValueTable(action_size= action_size,learning_rate=learning_rate)
agent_1 = BJA.BlackjackAgent(env, Q_value_table)
agent_2 = BJA.BlackjackAgent(env, Q_value_table)

experiences, rewards = agent_1.collect_experience(100)
# evalution = agent_1.evaluate(100)

# print(evalution)
Q_value_table.update(experiences)

# evalution = agent_2.evaluate(100)
# print(evalution)
evalution = agent_1.evaluate(100)
print("Linear training model Q-key size",len(Q_value_table.q_dict.keys()))
print("Linear training model with 100 experiences:",evalution)

# Loading Model from multiprocess training
Q_value_table_mp = QVT.QValueTable(action_size)
Q_value_table_mp.load()

agent_3 = BJA.BlackjackAgent(env, Q_value_table_mp)

evalution = agent_3.evaluate(100)
# 
print("MP model Q-key size",len(Q_value_table_mp.q_dict.keys()))
print("MP model",evalution)






Linear training model Q-key size 122
Linear training model with 100 experiences: {'avg_reward': -0.39, 'win_rate': 0.28, 'draw_rate': 0.05, 'loss_rate': 0.67}
MP model Q-key size 180
MP model {'avg_reward': -0.27, 'win_rate': 0.33, 'draw_rate': 0.07, 'loss_rate': 0.6}


In [8]:
# testing nested dict structure
'''outputs_of_processes := dict contained as 
    #<process_id: end_results_of_episodes_of_that_process>
        # <end_results_of_episodes_of_that_process : episode>
            # <episode: SAR outputs ~(experience, reward), ...>
                # episode: (experience, reward)
# '''
test_dict ={
    "PID 1": {
        1 :
        ("exp", "reward"),
        2 :
        ("exp", "reward")
    }
}
for proc_out in test_dict.values():
    print(proc_out)
    for episode in proc_out.values():
        # for result in episode:
        print(episode[0])




{1: ('exp', 'reward'), 2: ('exp', 'reward')}
exp
exp


In [9]:
import torch.multiprocessing as tmp

test_barrier = tmp.Barrier(1)
# def dummy_process(barrier):


print("Test Barrier status",test_barrier.broken)

print("passing thru test barrier")
test_barrier.wait()
# test_barrier.wait()

print("Passed barrier")
print("Test Barrier status",test_barrier.broken)
# print(test_barrier.broken)

test_barrier.reset()
print("Test Barrier status",test_barrier.broken)
print("Waiting at barrier again")
test_barrier.wait()
print("Test Barrier status",test_barrier.broken)



Test Barrier status False
passing thru test barrier
Passed barrier
Test Barrier status False
Test Barrier status False
Waiting at barrier again
Test Barrier status False


In [ ]:
from functools import partial
import gymnasium as gym
from blackjack_agent_package import blackjack_agent_class as BJA
from blackjack_agent_package import Q_value_table_class as QVT
import torch.multiprocessing as tmp

env = gym.make("Blackjack-v1", sab=False)
Q_value_table = QVT.QValueTable(action_size = env.action_space.n)
keyword_args = {
    "env" : env, 
    "q_values" : Q_value_table

}

def dummy_process(target_agent:BJA.BlackjackAgent):
    target_agent.collect_experience()
# print(BJA.BlackjackAgent.__init__)

agent = partial(BJA.BlackjackAgent, env = env, q_values = Q_value_table)
# agent.keywords = keyword_args
tmp.set_start_method('spawn',force=True) 
child_process = tmp.Process(target=dummy_process, kwargs={"target_agent": agent})

child_process.start()

child_process.join(10)


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/anaconda3/envs/GNC_Project_Env/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/anaconda3/envs/GNC_Project_Env/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'dummy_process' on <module '__main__' (built-in)>
